In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

num_input = 28
timesteps = 28
num_hidden = 128
num_classes = 10

X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [3]:
weight = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [4]:
def RNN(x, weights, biases):
    
    x = tf.unstack(x, timesteps, 1)
    
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [5]:
logits = RNN(X, weight, biases)
prediction = tf.nn.softmax(logits)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
with tf.Session() as sess:
    
    sess.run(init)
    
    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss = " + \
                 "{:.4f}".format(loss) + ", Training Accuracy= " + \
                 "{:.3f}".format(acc))
            
    print("Optimization Finished!")
    
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testintg Accuracy:", \
         sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss = 2.7202, Training Accuracy= 0.125
Step 200, Minibatch Loss = 2.1247, Training Accuracy= 0.266
Step 400, Minibatch Loss = 1.9290, Training Accuracy= 0.352
Step 600, Minibatch Loss = 1.6749, Training Accuracy= 0.477
Step 800, Minibatch Loss = 1.6111, Training Accuracy= 0.508
Step 1000, Minibatch Loss = 1.5627, Training Accuracy= 0.508
Step 1200, Minibatch Loss = 1.4143, Training Accuracy= 0.555
Step 1400, Minibatch Loss = 1.3310, Training Accuracy= 0.547
Step 1600, Minibatch Loss = 1.3015, Training Accuracy= 0.586
Step 1800, Minibatch Loss = 1.2902, Training Accuracy= 0.578
Step 2000, Minibatch Loss = 1.0914, Training Accuracy= 0.695
Step 2200, Minibatch Loss = 1.1287, Training Accuracy= 0.664
Step 2400, Minibatch Loss = 1.0486, Training Accuracy= 0.656
Step 2600, Minibatch Loss = 1.2107, Training Accuracy= 0.539
Step 2800, Minibatch Loss = 1.0564, Training Accuracy= 0.648
Step 3000, Minibatch Loss = 0.9970, Training Accuracy= 0.695
Step 3200, Minibatch Loss = 1.0